## Import Libraries
Load required packages for statistical analysis

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, pearsonr, spearmanr, mannwhitneyu, kruskal
import warnings

warnings.filterwarnings("ignore")

## Load Cleaned Data
Read the preprocessed TEDS-A 2023 dataset

In [4]:
df = pd.read_csv("1_datasets/processed/teds_analysis_ready.csv")

## Data Preparation for Statistical Tests
Create numeric mappings for categorical variables to enable correlation analysis

In [5]:
def map_to_numeric(series, mapping):
    return series.map(mapping)


categorical_mappings = {
    "age_group": {
        "12-14": 1,
        "15-17": 2,
        "18-20": 3,
        "21-24": 4,
        "25-29": 5,
        "30-34": 6,
        "35-39": 7,
        "40-44": 8,
        "45-49": 9,
        "50-54": 10,
        "55-64": 11,
        "65+": 12,
    },
    "service_type": {
        "Detox 24hr Hospital": 1,
        "Detox 24hr Residential": 2,
        "Rehab Hospital": 3,
        "Rehab Short-term (≤30 days)": 4,
        "Rehab Long-term (>30 days)": 5,
        "Intensive Outpatient": 6,
        "Non-intensive Outpatient": 7,
        "Outpatient Detox": 8,
    },
    "wait_time_days": {
        "Same day": 0,
        "1-7 days": 1,
        "8-14 days": 2,
        "15-30 days": 3,
        "31+ days": 4,
    },
    "prior_treatments": {
        "No prior": 0,
        "One prior": 1,
        "Two prior": 2,
        "Three prior": 3,
        "Four prior": 4,
        "Five or more prior": 5,
    },
    "employment_status": {
        "Full-time": 1,
        "Part-time": 2,
        "Unemployed": 3,
        "Not in labor force": 4,
    },
    "education_level": {
        "Less than Grade 9": 1,
        "Grades 9-11": 2,
        "Grade 12/GED": 3,
        "1-3 years college": 4,
        "4+ years college": 5,
    },
    "frequency_primary": {"No use past month": 1, "Some use": 2, "Daily use": 3},
    "recent_arrests": {"None": 0, "Once": 1, "Two or more": 2},
}

df_numeric = df.copy()
for col, mapping in categorical_mappings.items():
    if col in df_numeric.columns:
        df_numeric[col] = df_numeric[col].map(mapping)

## 1.1 Correlation: Wait Time vs Service Type
Test if different service types have different wait times using Spearman correlation


In [6]:
wait_service_data = df_numeric[["wait_time_days", "service_type"]].dropna()
corr_wait_service, p_wait_service = spearmanr(
    wait_service_data["wait_time_days"], wait_service_data["service_type"]
)

results_wait_service = {
    "correlation": corr_wait_service,
    "p_value": p_wait_service,
    "sample_size": len(wait_service_data),
}

## 1.2 Correlation: Prior Treatments vs Service Type
Examine if treatment history relates to service intensity using Spearman correlation

In [7]:
prior_service_data = df_numeric[["prior_treatments", "service_type"]].dropna()
corr_prior_service, p_prior_service = spearmanr(
    prior_service_data["prior_treatments"], prior_service_data["service_type"]
)

results_prior_service = {
    "correlation": corr_prior_service,
    "p_value": p_prior_service,
    "sample_size": len(prior_service_data),
}

## 1.3 Correlation: Age vs Years Using Substances
Analyze relationship between current age and duration of substance use

In [8]:
age_years_data = df_numeric[["age_group", "years_using"]].dropna()
corr_age_years, p_age_years = spearmanr(
    age_years_data["age_group"], age_years_data["years_using"]
)

results_age_years = {
    "correlation": corr_age_years,
    "p_value": p_age_years,
    "sample_size": len(age_years_data),
}

## 1.4 Correlation: Employment Status vs Treatment History
Assess relationship between employment and prior treatment episodes

In [9]:
employ_prior_data = df_numeric[["employment_status", "prior_treatments"]].dropna()
corr_employ_prior, p_employ_prior = spearmanr(
    employ_prior_data["employment_status"], employ_prior_data["prior_treatments"]
)

results_employ_prior = {
    "correlation": corr_employ_prior,
    "p_value": p_employ_prior,
    "sample_size": len(employ_prior_data),
}

## 1.5 Correlation: Substance Frequency vs Service Type
Test if frequency of substance use relates to treatment intensity

In [10]:
freq_service_data = df_numeric[["frequency_primary", "service_type"]].dropna()
corr_freq_service, p_freq_service = spearmanr(
    freq_service_data["frequency_primary"], freq_service_data["service_type"]
)

results_freq_service = {
    "correlation": corr_freq_service,
    "p_value": p_freq_service,
    "sample_size": len(freq_service_data),
}

## 1.6 Correlation Matrix: Key Continuous Variables
Calculate correlation matrix for numeric risk and treatment variables

In [11]:
continuous_vars = [
    "years_using",
    "number_of_substances",
    "age_group",
    "prior_treatments",
    "wait_time_days",
]
continuous_data = df_numeric[continuous_vars].dropna()
correlation_matrix = continuous_data.corr(method="spearman")

## 2.1 Chi-Square: Sex vs Primary Substance Category
Test if substance use patterns differ by sex

In [12]:
contingency_sex_substance = pd.crosstab(df["sex"], df["substance_category"])
chi2_sex_sub, p_sex_sub, dof_sex_sub, expected_sex_sub = chi2_contingency(
    contingency_sex_substance
)

results_sex_substance = {
    "chi2": chi2_sex_sub,
    "p_value": p_sex_sub,
    "degrees_of_freedom": dof_sex_sub,
    "cramers_v": np.sqrt(
        chi2_sex_sub
        / (
            contingency_sex_substance.sum().sum()
            * (min(contingency_sex_substance.shape) - 1)
        )
    ),
}

## 2.2 Chi-Square: Service Type vs Substance Category
Examine if different substances receive different treatment services

In [13]:
contingency_service_substance = pd.crosstab(
    df["service_type"], df["substance_category"]
)
chi2_serv_sub, p_serv_sub, dof_serv_sub, expected_serv_sub = chi2_contingency(
    contingency_service_substance
)

results_service_substance = {
    "chi2": chi2_serv_sub,
    "p_value": p_serv_sub,
    "degrees_of_freedom": dof_serv_sub,
    "cramers_v": np.sqrt(
        chi2_serv_sub
        / (
            contingency_service_substance.sum().sum()
            * (min(contingency_service_substance.shape) - 1)
        )
    ),
}

## 2.3 Chi-Square: Referral Source vs Service Type
Test if referral source affects type of treatment received

In [14]:
contingency_referral_service = pd.crosstab(df["referral_source"], df["service_type"])
chi2_ref_serv, p_ref_serv, dof_ref_serv, expected_ref_serv = chi2_contingency(
    contingency_referral_service
)

results_referral_service = {
    "chi2": chi2_ref_serv,
    "p_value": p_ref_serv,
    "degrees_of_freedom": dof_ref_serv,
    "cramers_v": np.sqrt(
        chi2_ref_serv
        / (
            contingency_referral_service.sum().sum()
            * (min(contingency_referral_service.shape) - 1)
        )
    ),
}

## 2.4 Chi-Square: Insurance Type vs Service Type
Analyze if insurance coverage affects treatment service access

In [15]:
contingency_insurance_service = pd.crosstab(df["health_insurance"], df["service_type"])
chi2_ins_serv, p_ins_serv, dof_ins_serv, expected_ins_serv = chi2_contingency(
    contingency_insurance_service
)

results_insurance_service = {
    "chi2": chi2_ins_serv,
    "p_value": p_ins_serv,
    "degrees_of_freedom": dof_ins_serv,
    "cramers_v": np.sqrt(
        chi2_ins_serv
        / (
            contingency_insurance_service.sum().sum()
            * (min(contingency_insurance_service.shape) - 1)
        )
    ),
}

## 2.5 Chi-Square: Mental Health Co-occurrence vs Substance Category
Test if co-occurring mental health varies by substance type

In [16]:
contingency_mh_substance = pd.crosstab(
    df["has_cooccurring_mental_health"], df["substance_category"]
)
chi2_mh_sub, p_mh_sub, dof_mh_sub, expected_mh_sub = chi2_contingency(
    contingency_mh_substance
)

results_mh_substance = {
    "chi2": chi2_mh_sub,
    "p_value": p_mh_sub,
    "degrees_of_freedom": dof_mh_sub,
    "cramers_v": np.sqrt(
        chi2_mh_sub
        / (
            contingency_mh_substance.sum().sum()
            * (min(contingency_mh_substance.shape) - 1)
        )
    ),
}

## 3.1 Mann-Whitney U Test: First-time vs Chronic Patients (Years Using)
Compare duration of substance use between treatment-naive and chronic patients

In [17]:
first_time_years = df_numeric[df_numeric["is_first_treatment"] == 1][
    "years_using"
].dropna()
chronic_years = df_numeric[df_numeric["is_chronic_treatment"] == 1][
    "years_using"
].dropna()

u_stat_years, p_years = mannwhitneyu(
    first_time_years, chronic_years, alternative="two-sided"
)

results_firsttime_chronic_years = {
    "u_statistic": u_stat_years,
    "p_value": p_years,
    "first_time_median": first_time_years.median(),
    "chronic_median": chronic_years.median(),
    "first_time_n": len(first_time_years),
    "chronic_n": len(chronic_years),
}

## 3.2 Mann-Whitney U Test: First-time vs Chronic Patients (Wait Time)
Compare wait times between treatment-naive and chronic patients

In [18]:
first_time_wait = df_numeric[df_numeric["is_first_treatment"] == 1][
    "wait_time_days"
].dropna()
chronic_wait = df_numeric[df_numeric["is_chronic_treatment"] == 1][
    "wait_time_days"
].dropna()

u_stat_wait, p_wait = mannwhitneyu(
    first_time_wait, chronic_wait, alternative="two-sided"
)

results_firsttime_chronic_wait = {
    "u_statistic": u_stat_wait,
    "p_value": p_wait,
    "first_time_median": first_time_wait.median(),
    "chronic_median": chronic_wait.median(),
    "first_time_n": len(first_time_wait),
    "chronic_n": len(chronic_wait),
}

## 3.3 Kruskal-Wallis Test: Wait Time Across Substance Categories
Test if wait times differ across different substance types

In [19]:
substance_groups = df_numeric.groupby("substance_category")["wait_time_days"].apply(
    list
)
substance_groups_clean = [
    np.array(group)[~np.isnan(group)] for group in substance_groups if len(group) > 0
]

h_stat_wait_substance, p_wait_substance = kruskal(*substance_groups_clean)

results_wait_substance_kruskal = {
    "h_statistic": h_stat_wait_substance,
    "p_value": p_wait_substance,
    "n_groups": len(substance_groups_clean),
}

## 3.4 Kruskal-Wallis Test: Number of Substances Across Service Types
Test if polysubstance use varies by treatment service type

In [20]:
service_groups = df_numeric.groupby("service_type")["number_of_substances"].apply(list)
service_groups_clean = [
    np.array(group)[~np.isnan(group)] for group in service_groups if len(group) > 0
]

h_stat_substances_service, p_substances_service = kruskal(*service_groups_clean)

results_substances_service_kruskal = {
    "h_statistic": h_stat_substances_service,
    "p_value": p_substances_service,
    "n_groups": len(service_groups_clean),
}

## 3.5 Mann-Whitney U Test: CJ-Referred vs Self-Referred (Prior Treatments)
Compare treatment history between criminal justice and self-referrals

In [21]:
cj_referred_prior = df_numeric[df_numeric["is_criminal_justice_referral"] == 1][
    "prior_treatments"
].dropna()
self_referred_prior = df_numeric[df["referral_source"] == "Self/Individual"][
    "prior_treatments"
].dropna()

u_stat_referral, p_referral = mannwhitneyu(
    cj_referred_prior, self_referred_prior, alternative="two-sided"
)

results_referral_prior = {
    "u_statistic": u_stat_referral,
    "p_value": p_referral,
    "cj_median": cj_referred_prior.median(),
    "self_median": self_referred_prior.median(),
    "cj_n": len(cj_referred_prior),
    "self_n": len(self_referred_prior),
}

## 3.6 Mann-Whitney U Test: Opioid vs Stimulant Users (Service Type)
Compare treatment intensity between opioid and stimulant primary users

In [22]:
opioid_service = df_numeric[df_numeric["is_opioid_primary"] == 1][
    "service_type"
].dropna()
stimulant_service = df_numeric[df_numeric["is_stimulant_primary"] == 1][
    "service_type"
].dropna()

u_stat_drug_service, p_drug_service = mannwhitneyu(
    opioid_service, stimulant_service, alternative="two-sided"
)

results_drug_service = {
    "u_statistic": u_stat_drug_service,
    "p_value": p_drug_service,
    "opioid_median": opioid_service.median(),
    "stimulant_median": stimulant_service.median(),
    "opioid_n": len(opioid_service),
    "stimulant_n": len(stimulant_service),
}

## 4.1 Point-Biserial: Homeless vs Years Using
Test relationship between homelessness and duration of substance use

In [23]:
homeless_years_data = df_numeric[["is_homeless", "years_using"]].dropna()
corr_homeless_years, p_homeless_years = pearsonr(
    homeless_years_data["is_homeless"], homeless_years_data["years_using"]
)

results_homeless_years = {
    "correlation": corr_homeless_years,
    "p_value": p_homeless_years,
    "sample_size": len(homeless_years_data),
}

## 4.2 Point-Biserial: Polysubstance Use vs Wait Time
Examine if polysubstance users experience different wait times

In [24]:
poly_wait_data = df_numeric[["is_polysubstance", "wait_time_days"]].dropna()
corr_poly_wait, p_poly_wait = pearsonr(
    poly_wait_data["is_polysubstance"], poly_wait_data["wait_time_days"]
)

results_poly_wait = {
    "correlation": corr_poly_wait,
    "p_value": p_poly_wait,
    "sample_size": len(poly_wait_data),
}

## 4.3 Point-Biserial: Mental Health Co-occurrence vs Prior Treatments
Test if co-occurring mental health relates to treatment history

In [25]:
mh_prior_data = df_numeric[["has_mental_health_disorder", "prior_treatments"]].dropna()
corr_mh_prior, p_mh_prior = pearsonr(
    mh_prior_data["has_mental_health_disorder"], mh_prior_data["prior_treatments"]
)

results_mh_prior = {
    "correlation": corr_mh_prior,
    "p_value": p_mh_prior,
    "sample_size": len(mh_prior_data),
}

## 4.4 Point-Biserial: Injection Use vs Service Type
Analyze if injection drug users receive different treatment services

In [26]:
idu_service_data = df_numeric[["is_injection_user", "service_type"]].dropna()
corr_idu_service, p_idu_service = pearsonr(
    idu_service_data["is_injection_user"], idu_service_data["service_type"]
)

results_idu_service = {
    "correlation": corr_idu_service,
    "p_value": p_idu_service,
    "sample_size": len(idu_service_data),
}

## 4.5 Point-Biserial: Criminal Justice Referral vs Number of Substances
Test if CJ-referred patients have different polysubstance patterns

In [27]:
cj_substances_data = df_numeric[
    ["is_criminal_justice_referral", "number_of_substances"]
].dropna()
corr_cj_substances, p_cj_substances = pearsonr(
    cj_substances_data["is_criminal_justice_referral"],
    cj_substances_data["number_of_substances"],
)

results_cj_substances = {
    "correlation": corr_cj_substances,
    "p_value": p_cj_substances,
    "sample_size": len(cj_substances_data),
}

## Create Comprehensive Results Summary
Organize all statistical test results into a structured dictionary

In [28]:
statistical_results = {
    "correlations": {
        "wait_time_vs_service_type": results_wait_service,
        "prior_treatments_vs_service_type": results_prior_service,
        "age_vs_years_using": results_age_years,
        "employment_vs_prior_treatments": results_employ_prior,
        "frequency_vs_service_type": results_freq_service,
        "correlation_matrix": correlation_matrix.to_dict(),
    },
    "chi_square_tests": {
        "sex_vs_substance": results_sex_substance,
        "service_type_vs_substance": results_service_substance,
        "referral_vs_service_type": results_referral_service,
        "insurance_vs_service_type": results_insurance_service,
        "mental_health_vs_substance": results_mh_substance,
    },
    "group_comparisons": {
        "firsttime_vs_chronic_years": results_firsttime_chronic_years,
        "firsttime_vs_chronic_wait": results_firsttime_chronic_wait,
        "wait_time_across_substances": results_wait_substance_kruskal,
        "substances_across_services": results_substances_service_kruskal,
        "cj_vs_self_referral": results_referral_prior,
        "opioid_vs_stimulant_service": results_drug_service,
    },
    "point_biserial": {
        "homeless_vs_years_using": results_homeless_years,
        "polysubstance_vs_wait_time": results_poly_wait,
        "mental_health_vs_prior_treatments": results_mh_prior,
        "injection_use_vs_service_type": results_idu_service,
        "cj_referral_vs_substances": results_cj_substances,
    },
}

## Print the outputs
Print all of the outputs of the analysis done above

In [ ]:
correlation_data = []
for name, res in statistical_results["correlations"].items():
    if name != "correlation_matrix":
        correlation_data.append(
            {
                "Variable_Pair": name,
                "Correlation": float(res["correlation"]),
                "P_Value": float(res["p_value"]),
                "Sample_Size": int(res["sample_size"]),
                "Significant": "Yes" if res["p_value"] < 0.05 else "No",
            }
        )

correlations_df = pd.DataFrame(correlation_data)

chi_data = []
for name, res in statistical_results["chi_square_tests"].items():
    chi_data.append(
        {
            "Test": name,
            "Chi2": float(res["chi2"]),
            "P_Value": float(res["p_value"]),
            "Cramers_V": float(res["cramers_v"]),
            "Significant": "Yes" if res["p_value"] < 0.05 else "No",
        }
    )

chi_df = pd.DataFrame(chi_data)

group_data = []
for name, res in statistical_results["group_comparisons"].items():
    test_stat = res.get("u_statistic") or res.get("h_statistic")
    group_data.append(
        {
            "Comparison": name,
            "Test_Statistic": float(test_stat),
            "P_Value": float(res["p_value"]),
            "Significant": "Yes" if res["p_value"] < 0.05 else "No",
        }
    )

group_df = pd.DataFrame(group_data)

pb_data = []
for name, res in statistical_results["point_biserial"].items():
    pb_data.append(
        {
            "Variable_Pair": name,
            "Correlation": float(res["correlation"]),
            "P_Value": float(res["p_value"]),
            "Sample_Size": int(res["sample_size"]),
            "Significant": "Yes" if res["p_value"] < 0.05 else "No",
        }
    )

pb_df = pd.DataFrame(pb_data)

print("CORRELATIONS:")
display(correlations_df)

print("\nCHI-SQUARE TESTS:")
display(chi_df)

print("\nGROUP COMPARISONS: ")
display(group_df)

print("\nPOINT BISERIAL CORRELATIONS:")
display(pb_df)

CORRELATIONS:


,Variable_Pair,Correlation,P_Value,Sample_Size,Significant
0,wait_time_vs_service_type,0.027264,3.314206e-120,730886,Yes
1,prior_treatments_vs_service_type,-0.163371,0.000000e+00,1156023,Yes
2,age_vs_years_using,0.755003,0.000000e+00,1212807,Yes
3,employment_vs_prior_treatments,0.144285,0.000000e+00,1122684,Yes
4,frequency_vs_service_type,-0.363586,0.000000e+00,1247350,Yes



CHI-SQUARE TESTS:


,Test,Chi2,P_Value,Cramers_V,Significant
0,sex_vs_substance,5768.773228,0.0,0.067414,Yes
1,service_type_vs_substance,115370.766466,0.0,0.134825,Yes
2,referral_vs_service_type,142964.944604,0.0,0.139799,Yes
3,insurance_vs_service_type,8648.537351,0.0,0.090902,Yes
4,mental_health_vs_substance,2604.622140,0.0,0.049849,Yes



GROUP COMPARISONS: 


,Comparison,Test_Statistic,P_Value,Significant
0,firsttime_vs_chronic_years,7.731263e+10,0.000000e+00,Yes
1,firsttime_vs_chronic_wait,2.772237e+10,9.101983e-258,Yes
2,wait_time_across_substances,2.474188e+03,0.000000e+00,Yes
3,substances_across_services,2.112439e+04,0.000000e+00,Yes
4,cj_vs_self_referral,6.258076e+10,0.000000e+00,Yes
5,opioid_vs_stimulant_service,5.565170e+10,0.000000e+00,Yes



POINT BISERIAL CORRELATIONS:


,Variable_Pair,Correlation,P_Value,Sample_Size,Significant
0,homeless_vs_years_using,0.059572,0.000000e+00,1212807,Yes
1,polysubstance_vs_wait_time,0.052226,0.000000e+00,730886,Yes
2,mental_health_vs_prior_treatments,0.137329,0.000000e+00,1156023,Yes
3,injection_use_vs_service_type,-0.004191,2.331889e-06,1269353,Yes
4,cj_referral_vs_substances,-0.006645,7.062188e-14,1269353,Yes
